In [1]:
! pip install transformers
! pip install datasets

In [2]:
cd '/content/drive/MyDrive/CAMemBERT2'

/content/drive/MyDrive/CAMemBERT2


In [3]:
from transformers import TrainingArguments,Trainer,AutoModelForSequenceClassification,AutoModel
import numpy as np
from math import floor
import torch
from torch.cuda import is_available
from collections import defaultdict
from scipy.stats import spearmanr
from sklearn.metrics import cohen_kappa_score,classification_report,fbeta_score
from models_f import MultiTaskModel
from trainer_f import MultiTaskModelTrainer
from trainer_utils import (training_args,training_kwargs,get_dataset,
                           freeze_layers,normalise_scores,create_params_grid)

In [ ]:
for pretrained_model_name in ['distilroberta-base']:
    model_type = 'roberta' if 'roberta' in pretrained_model_name else 'bert'
    model_n_layers = 6 if 'roberta' in pretrained_model_name else 12
    params_grid = create_params_grid({'tasks':[['aes','ged','ner']],
                                      'lr':[4e-5],
                                      'shared_encoder_n_layers':[0,1,2],
                                      'frozen_layers':[None],
                                      'output_layer':[2,4,-1]}
                                      )

    prev_task = None
    for i,params in enumerate(params_grid):
        print(params)
        if params['tasks']!=prev_task:
            dataset_obj,dataset_dict = get_dataset(params['tasks'],pretrained_model=pretrained_model_name,max_length=512)
            dataset_dict = dataset_dict.map(normalise_scores)

        trainer_kwargs,model_kwargs = training_kwargs(
            tasks=params['tasks'],
            class_weights={'aes':None,'ged':dataset_obj.get_weights(),'ner':None},
            early_stopping_patience=2,
            shared_encoder_n_layers=params['shared_encoder_n_layers'],
            output_layer = {'aes':-1,
                            'ged':params['output_layer'],
                            'ner':params['output_layer']},
            early_stopping_metric='los',
            normalised_values=[40,0],
            frozen_layers=params['frozen_layers'],
            save_results=True)

        args = training_args(lr=params['lr'],epochs=8)
        model = MultiTaskModel(kwargs_dict = model_kwargs,pretrained_model=pretrained_model_name)
        model = freeze_layers(model,frozen_layers=params['frozen_layers'],model_type=model_type)
        dev_set = 'dev' if params['tasks']!='ner' else 'validation'
        trainer = MultiTaskModelTrainer(
            model,
            args,
            train_dataset=dataset_dict['train'],
            eval_dataset=dataset_dict[dev_set],
        )
        trainer.kwargs_dict = trainer_kwargs
        trainer.test_dataset = dataset_dict['test']
        trainer.train()
        prev_task=params['tasks']